In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import hvplot.pandas
from scipy.stats import linregress


In [3]:
#Save hurricanes df to directory
hurricanes_df = pd.read_csv('Output_CSVs/hurricanes_df.csv')

#Save Merged df to directory
hpi_df = pd.read_csv('Output_CSVs/hpi_df.csv')

#Save clean df as CSV to directory
clean_hurricane_hpi_df = pd.read_csv('Output_CSVs/clean_hurricane_hpi_data.csv')
hpi_df.head()

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,Year -1,Year +1,Year +2,Year +3,Annual Change (%) -1,Annual Change (%) +1,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3
0,AL,Autauga,1001,1986,NaN,100.00,95.59,70.96,NaN,1987.0,1988.0,1989.0,NaN,-1.94,2.57,4.32,NaN,98.06,100.58,104.92
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59,1986.0,1988.0,1989.0,1990.0,NaN,2.57,4.32,-0.29,100.00,100.58,104.92,104.62
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38,1987.0,1989.0,1990.0,1991.0,-1.94,4.32,-0.29,2.52,98.06,104.92,104.62,107.25
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46,1988.0,1990.0,1991.0,1992.0,2.57,-0.29,2.52,3.09,100.58,104.62,107.25,110.57
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24,1989.0,1991.0,1992.0,1993.0,4.32,2.52,3.09,3.35,104.92,107.25,110.57,114.28


In [12]:
by_nation = hpi_df.groupby(['Year'])
change_by_nation = by_nation['Annual Change (%)'].agg(['mean','median'])
change_by_nation

,mean,median
Year,,
1975,NaN,NaN
1976,7.012998,7.490
1977,11.236533,10.670
1978,13.301912,13.600
1979,13.148394,12.480
1980,7.598505,7.450
1981,4.597900,5.010
1982,0.541563,0.950
1983,7.587261,5.725


In [13]:
by_state = hpi_df.groupby(['State','Year'])
change_by_state = by_state['Annual Change (%)'].agg(['mean','median'])
change_by_state

mean  median
State Year                   
AK    1975        NaN     NaN
      1976   7.550000   7.550
      1977   4.780000   4.780
      1978  11.830000  11.830
      1979  11.200000  11.200
...               ...     ...
WY    2019   3.885000   3.795
      2020   3.487727   3.625
      2021   9.633636   9.390
      2022  15.822727  15.125
      2023   4.909524   5.900

[2497 rows x 2 columns]